# The Solow Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import sympy as sm
import ipywidgets as widgets
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# The general solow model - definition

Consider the **standard Solow-model** where:

1. $K_t$ is capital
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

The Cobb Douglas function for the representative firm is given by:
$$
Y_t=K_t^\alpha (A_t L_t)^{1-\alpha}
$$

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$


when using sympy to fine the steady state, we have to solve the following:

$$ \tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] $$


In [2]:
# use sympy for defining symbols and functions

# define symbols
k = sm.symbols('k')
s = sm.symbols('s')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
n = sm.symbols('n')
g = sm.symbols('g')

# define function
f = sm.Function('f')(k)
f = k**alpha

# Define function to solve for steady state
ss = sm.Eq(k,(s*f + (1-delta)*k )/((1+n)*(1+g)))


The steady state can now be solved analytically

In [3]:
# solving for steady state analytically
ss_sol = sm.solve(ss,k)[0]
ss_sol


((delta + g*n + g + n)/s)**(1/(alpha - 1))

In [4]:
# Transformation into python function
ss_sol_func = sm.lambdify((s,alpha,delta,n,g),ss_sol)


We can now set parameter values and calculate the steady state

In [5]:
# Define parameters
s = 0.3
alpha = 0.3
delta = 0.1
n = 0.01
g = 0.02

In [6]:
# show the result of the steady state calculated analytically

print('The steady state is: ',ss_sol_func(s,alpha,delta,n,g))

The steady state is:  3.2951243994886896


# Calculation numerically

In stead of finding the steady state analytically, one can also do it numerically. This is done by creating a root finding problem, and then solve for the root

$$
0 = \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] - \tilde{k}^{\ast} 
$$

In [7]:
# Define parameters
s = 0.3
alpha = 0.3
delta = 0.1
n = 0.01
g = 0.02

# write your code here
f = lambda k: k**alpha
obj_kss = lambda kss: (s*f(kss) + (1-delta)*kss)/((1+g)*(1+n)) - kss
result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='brentq')

print('the steady state for k is', result.root)  

the steady state for k is 3.295124399488689


# Visualising the Solow model

In [8]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def cobb_douglas(K, L, A, alpha):
    return A * K**alpha * L**(1 - alpha)

def plot_solow_model(s, alpha, delta, n, g, T, variable):
    K0 = 1
    L0 = 1
    A0 = 1
    Y0 = 1
    Kt = 1

    K = [K0]
    L = [L0]
    A = [A0]
    Y = [Y0]
    Kt = [Kt]

    for t in range(T):
        Kt_new = s * cobb_douglas(K[t] / (A[t] * L[t]), 1, 1, alpha) + (1 - delta) * K[t] / (A[t] * L[t])
        K_new = Kt_new * A[t] * L[t]
        K.append(K_new)

        L_new = (1 + n) * L[t]
        L.append(L_new)

        A_new = (1 + g) * A[t]
        A.append(A_new)

        Y_new = cobb_douglas(K_new, L_new, A_new, alpha)
        Y.append(Y_new)

        Kt.append(Kt_new)

    if variable == 'K':
        plt.plot(K)
        plt.title('Plot of K')
        plt.ylabel('K')
    elif variable == 'Y':
        plt.plot(Y)
        plt.title('Plot of Y')
        plt.ylabel('Y')
    elif variable == 'K/A*L':
        K_AL = [K[i] / (A[i] * L[i]) for i in range(len(K))]
        plt.plot(K_AL)
        plt.title('Plot of K/(A*L)')
        plt.ylabel('K/(A*L)')

    plt.xlabel('Time')
    plt.show()

widgets.interact(
    plot_solow_model,
    s=widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='s'),
    alpha=widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='alpha'),
    delta=widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01, description='delta'),
    n=widgets.FloatSlider(value=0.01, min=0, max=0.1, step=0.001, description='n'),
    g=widgets.FloatSlider(value=0.02, min=0, max=0.1, step=0.001, description='g'),
    T=widgets.IntSlider(value=100, min=10, max=1000, step=10, description='T'),
    variable=widgets.Dropdown(
        options=['K', 'Y', 'K/A*L'],
        value='K',
        description='Variable:'
    )
)


interactive(children=(FloatSlider(value=0.3, description='s', max=1.0, step=0.01), FloatSlider(value=0.3, desc…

<function __main__.plot_solow_model(s, alpha, delta, n, g, T, variable)>

We add oil and land to the model to add more dimensions to the model

Now the production function for the representative firm is given by:

$$ 
Y_t=K_t^\alpha (A_t L_t)^\beta X^\kappa E_t^\epsilon, \hspace{5mm}    \alpha>0, \beta>0, \kappa>0, \epsilon>0, \hspace{5mm}  \alpha+\beta+\kappa+\epsilon=1
$$

Where $X$ is land and $E_t$ is oil

$$
L_t=(1+n)L_t, \hspace{5mm}   n\geq0
$$

$$
A_{t+1}=(1+g)A_t, \hspace{5mm} g\geq0
$$

The depletion of the natural ressource is described by: 
$$
R_{t+1}=R_t-E_t
$$

Where $R_t$ is the remaining stock of oil in each period and $E_t$ is how much is oil is consumed.

the oil consumption is given as:

$$
E_t=s_E R_t , \hspace{5mm} 0<s_E<1
$$

**Saving** is still a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t},X,E_t)+(1-\delta)K_{t}, \delta \in (0,1)
$$

Inserting $E_t=s_E R_t$ into the production function and dividing both sides by $L_t$ yields

$$
y_t=k_t^\alpha A_t^\beta (\frac{X}{L_t})^\kappa (\frac{s_E R_t}{L_t})^\epsilon = s_E^\epsilon X^\kappa k_t^\alpha A_t^\beta R_t^\epsilon L_t^{-\kappa-\epsilon}
$$

By taking logs we get the following:

$$
g_t^y=\alpha g_t^k+\beta g_t^A + \epsilon g_t^R - (\kappa+\epsilon)g_t^L \approx \alpha g_t^k +\beta g - \epsilon s_E - (\kappa+\epsilon)n
$$

As $g_t^y$ and $g_t^k$ converge towards the same rate they are set equal. This yields:

$$
g^y=\frac{\beta}{\beta+\kappa+\epsilon}g-\frac{\kappa + \epsilon}{\beta+\kappa+\epsilon}n - \frac{\epsilon}{\beta+\kappa+\epsilon}s_E
$$

## Solow model with land and oil

In [81]:
import numpy as np

class SolowModel:

    def __init__(self, T, s, s_E, n, g, delta, alpha, beta, kappa, epsilon):
        self.T = T
        self.s = s
        self.s_E = s_E
        self.n = n
        self.g = g
        self.delta = delta
        self.alpha = alpha
        self.beta = beta
        self.kappa = kappa
        self.epsilon = epsilon

    def simulate(self):
        T = self.T
        A = np.zeros(T)
        K = np.zeros(T)
        L = np.zeros(T)
        X = 1
        R = np.zeros(T)
        E = np.zeros(T)
        Y = np.zeros(T)

        A[0] = 1
        K[0] = 1
        L[0] = 1
        R[0] = 1
        E[0] = self.s_E * R[0]
        Y[0] = K[0]**self.alpha * (A[0] * L[0])**self.beta * X**self.kappa * E[0]**self.epsilon


        for t in range(1, T):
            A[t] = A[t-1] * (1 + self.g)
            L[t] = L[t-1] * (1 + self.n)
            R[t] = R[t-1] - E[t-1]
            E[t] = self.s_E * R[t-1]
            Y[t] = K[t-1]**self.alpha * (A[t] * L[t])**self.beta * X**self.kappa * E[t]**self.epsilon
            K[t] = self.s * Y[t-1] + (1 - self.delta) * K[t-1]


        return A, K, L, X, R, E, Y


In [84]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

def plot_solow_model(T, s, s_E, n, g, delta, alpha, beta, kappa, epsilon, variable):
    model = SolowModel(T, s, s_E, n, g, delta, alpha, beta, kappa, epsilon)
    A, K, L, X, R, E, Y = model.simulate()

    t = np.arange(T)
    plt.figure(figsize=(8, 6))

    if variable == 'Y':
        plt.plot(t, Y, label="Output (Y)")
        plt.ylabel("Output")
    elif variable == 'K':
        plt.plot(t, K, label="Capital per capita (K)")
        plt.ylabel("Capital per capita")
    elif variable == 'K/(A*L)':
        plt.plot(t, K / (A * L), label="Capital per effective worker (K/(A*L))")
        plt.ylabel("Capital per effective worker")

    plt.xlabel("Time")
    plt.legend()
    plt.show()

# Sliders and dropdown menu
T_slider = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="T")
s_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.2, description="s")
s_E_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.1, description="s_E")
n_slider = widgets.FloatSlider(min=0.0, max=0.1, step=0.01, value=0.02, description="n")
g_slider = widgets.FloatSlider(min=0.0, max=0.1, step=0.01, value=0.02, description="g")
delta_slider = widgets.FloatSlider(min=0.0, max=0.1, step=0.01, value=0.1, description="delta")
alpha_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.3, description="alpha")
beta_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.2, description="beta")
kappa_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.2, description="kappa")
epsilon_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.3, description="epsilon")
variable_dropdown = widgets.Dropdown(options=['Y', 'K', 'K/(A*L)'], value='Y', description='Variable')

# Interactive plot
widgets.interact(plot_solow_model,
                 T=T_slider,
                 s=s_slider,
                 s_E=s_E_slider,
                 n=n_slider,
                 g=g_slider,
                 delta=delta_slider,
                 alpha=alpha_slider,
                 beta=beta_slider,
                 kappa=kappa_slider,
                 epsilon=epsilon_slider,
                 variable=variable_dropdown)


interactive(children=(IntSlider(value=100, description='T', max=1000, min=10, step=10), FloatSlider(value=0.2,…

<function __main__.plot_solow_model(T, s, s_E, n, g, delta, alpha, beta, kappa, epsilon, variable)>

# Visualising the difference between the models

The Green Solow Model extends the traditional Solow Model by incorporating the impact of emissions and abatement efforts on the economy. The model introduces an additional equation:

\begin{align}
E_t=\Omega Y_t-\Omega A(Y_t,R_t) \ \ (1)
\end{align}

where $E_t$ represents total emissions, $Y_t$ is the production output, and $A(Y_t, R_t)$ is the abatement function. The parameter $\Omega$ captures the proportion of production that contributes to emission costs. Emissions are directly proportional to production and are reduced by the abatement efforts.

The abatement function $A(Y_t, R_t)$ is assumed to be increasing in output, $Y_t$, and abatement efforts, $R_t$, but with diminishing returns. Abatement efforts come with a cost, reducing the overall output:

\begin{align}
Y=Y_t-R_t=(1-\theta)Y_t \ \ (2)
\end{align}

By rewriting the equation for emissions, we can better understand its relationship to output and abatement efforts:

\begin{align}
E=\Omega (Y_t-A_t(Y_t,R_t)) \quad \quad (3) \\
E=\Omega Y_t(1-A_t(1,\frac{R_t}{Y_t}))  \quad \quad (4) \\
E=\Omega Y_t \alpha(\theta_t), \theta_t \equiv \frac{R_t}{Y_t}  \alpha(\theta_t) \equiv 1-A_t(1,\theta_t) \quad \quad(5)
\end{align}

The production function still exhibits constant returns to scale. Consequently, we can rewrite the model in terms of output per effective worker:

\begin{align}
Y_t=K_t^\alpha (A_tL_t)^{1-\alpha} \Rightarrow y_t=k_t^\alpha y_t\equiv \frac{Y_t}{A_tL_t}, k_t\equiv \frac{K_t}{A_tL_t} \quad \quad (6)
\end{align}

With this, we can derive the new transition equation for capital per effective worker:

\begin{align}
\dot{k_t}=s(1-\theta_t)k_t^\alpha-k_t(\delta+g+n) \quad \quad (7)
\end{align}

This model allows us to study the impact of emissions, abatement efforts, and their associated costs on capital accumulation, output, and long-term economic growth.

The steady state is found where
\begin{align}
0=s(1-\theta_t)k_t^\alpha-k_t(\delta+g+n) \quad \quad (8)
\end{align}

We calculate this numerically

In [12]:
# Define parameters
s_g = 0.3
alpha_g = 0.3
delta_g = 0.1
n_g = 0.01
g_g = 0.02
theta_g = 0.05

# solve for steady state
f = lambda k_g: k_g**alpha
obj_kss = lambda kss: s_g * (1-theta_g) * f(kss) - (n_g + g_g + delta_g) * kss
result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='brentq')

print('the steady state for k is', result.root)  


the steady state for k is 3.069037209074208


## Visualisation of the green solow model



In [24]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Function to calculate and display steady-state capital
def calculate_steady_state(alpha, delta, g, n, theta, s):
    k_star = ((delta + g + n) / (s * (1 - theta)))**(1 / (alpha - 1))
    print(f'Steady-state capital per capita (k_star): {k_star}')

# Create sliders for each parameter
alpha_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description=r'$\alpha$')
delta_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01, description=r'$\delta$')
g_slider = widgets.FloatSlider(value=0.02, min=0, max=1, step=0.01, description='$g$')
n_slider = widgets.FloatSlider(value=0.01, min=0, max=1, step=0.01, description='$n$')
theta_slider = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description=r'$\theta$')
s_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='$s$')

# Create an interactive widget
widgets.interactive(
    calculate_steady_state,
    alpha=alpha_slider,
    delta=delta_slider,
    g=g_slider,
    n=n_slider,
    theta=theta_slider,
    s=s_slider
)





interactive(children=(FloatSlider(value=0.3, description='$\\alpha$', max=1.0, step=0.01), FloatSlider(value=0…